<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8%20%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ast

# Загрузка данных
df = pd.read_excel("merged_with_matches_and_migrant_data.xlsx")

# Функция для проверки строки по всем трём условиям
def check_row(row):
    try:
        # Преобразуем строки в списки
        disciplines = ast.literal_eval(row["discipline"])
        checks = ast.literal_eval(row["check"])
        stages = ast.literal_eval(row["stages"])

        subjects = [s.strip() for s in str(row["Предмет"]).split(",")]
        subject_grades = [int(s.strip()) for s in str(row["Минимальный класс, где встречается"]).split(",")]

        # Условие 1: хотя бы один 'term' в check
        if "term" not in checks:
            return False

        # Составляем список дисциплин, где check == 'term'
        term_disciplines = [d for d, c in zip(disciplines, checks) if c == "term"]
        term_stages = [s for s, c in zip(stages, checks) if c == "term"]

        # Условие 2: совпадают названия предметов
        if set(term_disciplines) != set(subjects):
            return False

        # Условие 3: классы различаются не больше чем на 1
        for td in term_disciplines:
            if td not in subjects:
                return False
            idx_subject = subjects.index(td)
            idx_term = term_disciplines.index(td)
            subj_grade = subject_grades[idx_subject]
            term_grade = int(term_stages[idx_term])

            if abs(subj_grade - term_grade) > 1:
                return False

        return True

    except Exception:
        return False

# Применяем фильтрацию
filtered_df = df[df.apply(check_row, axis=1)]

# Сохраняем результат
filtered_df.to_excel("filtered_terms_by_conditions.xlsx", index=False)

print("✅ Файл filtered_terms_by_conditions.xlsx сохранён.")

✅ Файл filtered_terms_by_conditions.xlsx сохранён.


In [2]:
import pandas as pd
import ast

# Загрузка данных
df = pd.read_excel("merged_with_matches_and_migrant_data.xlsx")

def check_grade_difference(row):
    try:
        # Получаем дисциплины и классы из столбцов discipline и stages
        disciplines = ast.literal_eval(row["discipline"])
        discipline_grades = [int(s) for s in ast.literal_eval(row["stages"])]

        # Получаем предметы и классы из столбцов Предмет и Минимальный класс, где встречается
        subjects = [s.strip() for s in str(row["Предмет"]).split(",")]
        subject_grades = [int(s.strip()) for s in str(row["Минимальный класс, где встречается"]).split(",")]

        # Преобразуем в словари для сопоставления по названиям
        subj_grade_dict = dict(zip(subjects, subject_grades))
        disc_grade_dict = dict(zip(disciplines, discipline_grades))

        # Находим общие предметы
        common_subjects = set(subjects).intersection(set(disciplines))

        if not common_subjects:
            return False

        # Проверяем разницу в классах
        for subj in common_subjects:
            if abs(subj_grade_dict[subj] - disc_grade_dict[subj]) > 1:
                return False

        return True

    except Exception:
        return False

# Фильтруем строки
filtered_df = df[df.apply(check_grade_difference, axis=1)]

# Сохраняем результат
filtered_df.to_excel("filtered_by_class_difference.xlsx", index=False)

print("✅ Файл filtered_by_class_difference.xlsx сохранён.")

✅ Файл filtered_by_class_difference.xlsx сохранён.


In [4]:
import pandas as pd
import ast

# Загружаем файл
df = pd.read_excel("merged_with_matches_and_migrant_data.xlsx")

# Функция для фильтрации строк
def check_conditions(row):
    try:
        # Проверка 1: нет ни одного 'term'
        check_values = ast.literal_eval(str(row["check"]))
        if "term" in check_values:
            return False

        # Предметы из discipline
        discipline_subjects = ast.literal_eval(str(row["discipline"]))

        # Предметы из столбца "Предмет"
        if pd.isna(row["Предмет"]):
            predmets = []
        else:
            raw_items = str(row["Предмет"]).split(",")
            predmets = [s.strip() for s in raw_items if s.strip().lower() not in ('', 'nan')]

        # Объединяем и считаем уникальные предметы
        total_unique_subjects = set(discipline_subjects + predmets)

        # Проверка 2: должно быть 3 или больше уникальных предметов
        return len(total_unique_subjects) >= 3

    except Exception:
        return False

# Применяем фильтр
filtered_df = df[df.apply(check_conditions, axis=1)]

# Сохраняем результат
filtered_df.to_excel("merged_filtered_no_term_and_3plus_subjects.xlsx", index=False)

print("Файл 'merged_filtered_no_term_and_3plus_subjects.xlsx' сохранён.")

Файл 'merged_filtered_no_term_and_3plus_subjects.xlsx' сохранён.
